In [1]:
### import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()
# Country Data COUNTRY table
COUNTRY_df = pd.read_sql_query("select * from COUNTRY;", conn)
# League data from LEAGUE
LEAGUE_df = pd.read_sql_query("select * from LEAGUE;", conn)
#
Team_df = pd.read_sql_query("select * from TEAM;", conn)
# Match data from Matches - just pulling 1 year for now -2015/2016 might be latest year
# You can remove restriction if you want
#MATCH_df = pd.read_sql_query("select * from MATCH where SEASON = '2015/2016';", conn)
# Match data from Matches - just pulling 1 and relevant data for now - we can remove restriction later if we want
MATCH_df = pd.read_sql_query("SELECT DATE, LEAGUE_ID, HOME_TEAM_API_ID, AWAY_TEAM_API_ID, HOME_TEAM_GOAL, AWAY_TEAM_GOAL FROM MATCH WHERE SEASON = '2015/2016' ;",
conn)

### We calculate the "differential" for both the home team and away team

# FIND MOST DOMINATE BY 

MATCH_df["HOME_DIFF"] = MATCH_df["home_team_goal"]- MATCH_df["away_team_goal"]
MATCH_df["AWAY_DIFF"] = MATCH_df["away_team_goal"]- MATCH_df["home_team_goal"]

MATCH_home_df=MATCH_df.loc[:,["date","home_team_api_id", "HOME_DIFF","league_id","away_team_api_id"]]
MATCH_away_df=MATCH_df.loc[:,["date","away_team_api_id", "AWAY_DIFF","league_id","home_team_api_id"]]

MATCH_away_df.head()
MATCH_home_df.head()



MATCH_away_df.head()


# MATCH_away_df['W'] = '0' 
# MATCH_away_df['L'] = '0' 
# MATCH_away_df['T'] = '0' 
# MATCH_home_df['W'] = '0' 
# MATCH_home_df['L'] = '0' 
# MATCH_home_df['T'] = '0' 


# MATCH_away_df.loc[MATCH_away_df.AWAY_DIFF >0, 'W'] = '1' 
# MATCH_away_df.loc[MATCH_away_df.AWAY_DIFF <0, 'L'] = '1' 
# MATCH_away_df.loc[MATCH_away_df.AWAY_DIFF ==0, 'T'] = '1' 

# MATCH_home_df.loc[MATCH_home_df.HOME_DIFF >0, 'W'] = '1' 
# MATCH_home_df.loc[MATCH_home_df.HOME_DIFF <0, 'L'] = '1' 
# MATCH_home_df.loc[MATCH_home_df.HOME_DIFF ==0, 'T'] = '1' 

# MATCH_away_df.head(5)

MATCH_away_df['W'] = 0 
MATCH_away_df['L'] = 0 
MATCH_away_df['T'] = 0 
MATCH_home_df['W'] = 0 
MATCH_home_df['L'] = 0 
MATCH_home_df['T'] = 0 


MATCH_away_df.loc[MATCH_away_df.AWAY_DIFF >0, 'W'] = 1 
MATCH_away_df.loc[MATCH_away_df.AWAY_DIFF <0, 'L'] = 1 
MATCH_away_df.loc[MATCH_away_df.AWAY_DIFF ==0, 'T'] = 1 

MATCH_home_df.loc[MATCH_home_df.HOME_DIFF >0, 'W'] = 1 
MATCH_home_df.loc[MATCH_home_df.HOME_DIFF <0, 'L'] = 1 
MATCH_home_df.loc[MATCH_home_df.HOME_DIFF ==0, 'T'] = 1 

MATCH_away_df.head(5)



### We combine the home team data with the away team data to create 1 liast of games, with the differential

MATCH_home_df = MATCH_home_df.rename(columns={"home_team_api_id": "team_api_id","HOME_DIFF":"DIFF","away_team_api_id": "opponent" })
MATCH_away_df = MATCH_away_df.rename(columns={"away_team_api_id": "team_api_id","AWAY_DIFF":"DIFF","home_team_api_id": "opponent" })




frames = [MATCH_home_df, MATCH_away_df]
all_games_unsorted = pd.concat(frames)
all_games = all_games_unsorted.sort_values(by=["team_api_id","date"])
all_games.reset_index(inplace=True)
all_games.head() 

### We want to figure out the what the differential was for the next game as well.  
### We will make a copy of the table, and add "ng_" as a suffix to the columns for "next game"

next_game_unsorted = all_games.copy()
next_game_unsorted = next_game_unsorted.rename(columns={"DIFF": "ng_DIFF",
                                      "team_api_id": "ng_team_api_id2",
                                      "league_id":   "ng_league_id",
                                      "date":   "ng_date",
                                      "W": "ng_W",
                                      "L": "ng_L",
                                      "T": "ng_T" })

next_game = next_game_unsorted.sort_values(by=["ng_team_api_id2","ng_date"])
next_game.head()

### We will merge the original data to the copy, but shift the copy data by one row, so it will line up with the next game

#df_complete = pd.merge(all_games, next_game.shift(-1).fillna(0).astype(int), left_index=True, right_index=True)
df_complete = pd.merge(all_games, next_game.shift(-1).fillna(0), left_index=True, right_index=True)
df_complete.head(5)

### Oops!  For the last game of the season (where we don't have a next game), checking the next game would have returned the next team's first game.  We should just zero that out.      

#df.loc[df.First_name == 'Bill', 'name_match'] = 'Match' 
df_complete.sort_values(by=["team_api_id","date"]).head()
df_complete.loc[df_complete.team_api_id != df_complete.ng_team_api_id2, "ng_DIFF"] = 0
df_complete.loc[df_complete.team_api_id != df_complete.ng_team_api_id2, "ng_league_id"] = 0
df_complete.loc[df_complete.team_api_id != df_complete.ng_team_api_id2, "opponent_y"] = 0
df_complete.loc[df_complete.team_api_id != df_complete.ng_team_api_id2, "ng_date"] = 0
df_complete.loc[df_complete.team_api_id != df_complete.ng_team_api_id2, "ng_W"] = 0
df_complete.loc[df_complete.team_api_id != df_complete.ng_team_api_id2, "ng_L"] = 0
df_complete.loc[df_complete.team_api_id != df_complete.ng_team_api_id2, "ng_T"] = 0



df_complete.head(2)

### Let's delete all the extra columns we created

df_complete.drop(['index_x','index_y','ng_date','ng_league_id'], axis=1, inplace=True)
df_complete.head()

# df_complete rules!  

### Now, lets find the most dominant team.  We will do that by grouping by the bext team


df_teams = df_complete.groupby(['league_id','team_api_id'], as_index=False).sum()


# df_teams = df_complete.groupby(['league_id','team_api_id']).sum()
df_teams.head(5)
#df_teams.drop(['opponent_x','ng_team_api_id2','opponent_y'], axis=1, inplace=True)


#df_teams.head(50)
# Groups to get season DIFF and ng_DIFF and all teams so 


### For each league, we will append the team that has the maximum Differential.  These are our dominant teams

leagues = df_teams.league_id.unique()
x = []
for l in leagues:
    df_test = df_teams[df_teams.league_id == l]
    x.append(df_test['team_api_id'].loc[df_test['DIFF'].idxmax()])
print(x)

#df_complete.opponent_x.unique()


dom_match = df_complete[df_complete.opponent_x.isin(x)]
dom_match.head()

#dom_match.opponent_y.value_counts()

df_teams.head()



df_DIFF = df_teams[["team_api_id","DIFF"]]
df_DIFF = df_DIFF.rename(columns={"DIFF": "total_DIFF"})
df_DIFF.head(5)

dom_match.head()

# Get the season DIFF for the 
merge_table_all = pd.merge(dom_match, df_DIFF,left_on="team_api_id", right_on="team_api_id")
 
merge_table_c = merge_table_all.rename(columns={"opponent_x": "dom_team",
                                          "team_api_id": "team",
                                      "opponent_y": "ng_opponent",
                                      "total_DIFF":   "team_season_DIFF"                                  
                                                                     })


merge_table_c.head()

merge_table_b = pd.merge(merge_table_c, df_DIFF,left_on="dom_team", right_on="team_api_id")
merge_table_b = merge_table_b.rename(columns={"total_DIFF": "dom_season_DIFF"})
merge_table_b.drop(['team_api_id'], axis=1, inplace=True)
merge_table_b.head(5)


merge_table = pd.merge(merge_table_b, df_DIFF,left_on="ng_opponent", right_on="team_api_id")
merge_table = merge_table.rename(columns={"total_DIFF": "ng_opp_season_DIFF"})
merge_table.head(5)


merge_table.drop(['team_api_id'], axis=1, inplace=True)

#merge_table.value_count("ng_opponent")
#merge_table.ng_opponent.value_counts()
merge_table = merge_table.rename(columns={"ng_team_api_id2": "shift_team",
                                         "league_id":   "league",
                                       "ng_opponent":   "ng_oppo",
                                       "team_season_DIFF": "team_szn_DIFF",
                                       "dom_season_DIFF":  "dom_szn_DIFF",
                                       "ng_opp_season_DIFF": "ng_oppo_szn_DIFF"  }) 




merge_table

# df_teams.head() has the 
df_teams_WLT = df_teams[["team_api_id","W","L","T"]].copy()
#df_teams_WLT["win_%"]=df_teams_WLT[['T']] + df_teams_WLT[['L']]
#+ df_teams_WLT.L + df_teams_WLT.L
#df_teams_WLT.head()
#df_teams_WLT[["win_percent"]]=df_teams_WLT["W"


df_teams_WLT['total_games'] = df_teams_WLT.apply(lambda x: x['W'] + x['L'] + x['T'], axis=1)
df_teams_WLT['win_rate'] = df_teams_WLT.apply(lambda x: x['W'] / x['total_games'], axis=1)

df_teams_WLT.head()
                                          

merge_table.head()

### DATA TABLES

df_teams_WLT 
Has the total W/L/T and winning percentage of each team

merge_table
Has all the rows 

Note: 
shift_team is the team we merged on when we shifted.  If it's not the same as team it is the last game of the season

# scatter_x = merge_table["team_season_DIFF"]
# scatter_y = merge_table["ng_DIFF"]
# group = merge_table["league_id"]
# cdict = {0:"red",1:"orange",2:"blue",3:"green",4:"yellow",5:"black",6:"cyan",7:"magenta",8:"brown",9:"lime",10:"pink"}
# fig, ax = plt.subplots()
# for g in np.unique(group):
#     ix = np.where(group == g)
#     ax.scatter(scatter_x[ix], scatter_y[ix], c = cdict[g], label = g, s = 100)
# ax.legend()
# plt.show()

plt.xlabel("Team's Season Diff")
plt.ylabel("Team's Next Game Diff")
plt.axhline(0, color='b')
plt.axvline(0, color='b')
plt.scatter(merge_table["team_szn_DIFF"], merge_table["ng_DIFF"], marker="o", facecolors="red", edgecolors="black",
            alpha=0.75)




plt.xlabel("Team's Season Diff")
plt.ylabel("Team's Game Diff vs. Dominant Team")
plt.axhline(0, color='b')
plt.axvline(0, color='b')
plt.scatter(merge_table["team_szn_DIFF"], merge_table["DIFF"], marker="o", facecolors="red", edgecolors="black",
            alpha=0.75)


merge_table["dom_team"].unique()
#dom_team has 11 elements and dom_DIFF only has 9 elements

#dom_DIFF = merge_table["dom_season_DIFF"].unique()
#plt.bar(dom_team, dom_DIFF, color='r', alpha=0.5, align="center")


